In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error, mean_absolute_error, f1_score, confusion_matrix, precision_score, roc_curve, auc
from scipy.signal import savgol_filter
from tqdm import tqdm
from PyEMD import EEMD
import time

plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
sns.set(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)


class Config:
    symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'NVDA', 'TSLA']
    start_date = '2012-01-01'
    end_date = pd.Timestamp.now().strftime('%Y-%m-%d')
    seq_length = 20
    train_ratio = 0.8
    batch_size = 128
    epochs = 200
    lr = 5e-4
    weight_decay = 1e-4
    dropout = 0.2
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_config = {
        'Transformer': {'d_model': 128, 'nhead': 8, 'num_layers': 3},
        'Informer': {'d_model': 128, 'nhead': 8, 'num_layers': 3}
    }


class DataProcessor:
    def __init__(self):
        self.scaler = StandardScaler()

    def get_data(self, symbol):
        max_retries = 3
        for attempt in range(max_retries):
            try:
                df = yf.download(symbol, start=Config.start_date, end=Config.end_date)
                return df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
            except yf.YFRateLimitError:
                print(f"Rate limit reached for {symbol}. Retrying in 60 seconds...")
                time.sleep(60)
        print(f"Failed to download data for {symbol} after {max_retries} attempts.")
        return None

    def compute_rsi(self, prices, period=14):
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    def compute_macd(self, prices, fast=12, slow=26, signal=9):
        ema_fast = prices.ewm(span=fast, adjust=False).mean()
        ema_slow = prices.ewm(span=slow, adjust=False).mean()
        macd = ema_fast - ema_slow
        signal_line = macd.ewm(span=signal, adjust=False).mean()
        return macd, signal_line

    def add_features(self, df):
        df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
        df['MA5'] = df['Close'].rolling(5).mean()
        df['MA20'] = df['Close'].rolling(20).mean()
        df['RSI'] = self.compute_rsi(df['Close'])
        macd, signal = self.compute_macd(df['Close'])
        df['MACD'] = macd
        df['MACD_Signal'] = signal
        df['DayOfWeek'] = df.index.dayofweek
        df['Month'] = df.index.month - 1
        return df.dropna()

    def decompose_with_eemd(self, series):
        eemd = EEMD()
        T = np.arange(len(series))
        imfs = eemd.emd(series, T)
        return imfs

    def plot_eemd_decomposition(self, symbol, series, imfs):
        plt.figure(figsize=(12, 8))
        plt.subplot(len(imfs) + 1, 1, 1)
        plt.plot(series, label='series')
        plt.title(f'{symbol} EEMD')
        plt.legend()

        for i, imf in enumerate(imfs):
            plt.subplot(len(imfs) + 1, 1, i + 2)
            plt.plot(imf, label=f'IMF_{i}')
            plt.legend()

        plt.tight_layout()
        plt.savefig(f'{symbol}_eemd_decomposition.png')
        plt.close()

    def process_data(self, symbol):
        df = self.get_data(symbol)
        if df is None:
            return None, None
        df = self.add_features(df)
        window_length = min(11, len(df['LogReturn']))
        df['LogReturn'] = savgol_filter(df['LogReturn'], window_length, 2)

        log_return_series = df['LogReturn'].values
        imfs = self.decompose_with_eemd(log_return_series)
        self.plot_eemd_decomposition(symbol, log_return_series, imfs)

        for i, imf in enumerate(imfs):
            df[f'IMF_{i}'] = imf

        features = ['LogReturn', 'MA5', 'MA20', 'RSI', 'MACD', 'MACD_Signal', 'DayOfWeek', 'Month']
        for i in range(len(imfs)):
            features.append(f'IMF_{i}')
        target = 'LogReturn'
        df[features] = self.scaler.fit_transform(df[features])

        X, y = [], []
        for i in range(len(df) - Config.seq_length):
            X.append(df[features].values[i:i + Config.seq_length])
            y.append(df[target].values[i + Config.seq_length])
        return np.array(X), np.array(y)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dropout, use_positional_encoding=True):
        super().__init__()
        self.encoder = nn.Linear(input_size, d_model)
        self.use_positional_encoding = use_positional_encoding
        if use_positional_encoding:
            self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model, nhead, dim_feedforward=d_model * 4, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

        decoder_layer = nn.TransformerDecoderLayer(
            d_model, nhead, dim_feedforward=d_model * 4, dropout=dropout, batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.decoder = nn.Linear(d_model, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.encoder(x)
        if self.use_positional_encoding:
            x = self.pos_encoder(x)
        memory = self.transformer_encoder(x)

        tgt = x
        if self.use_positional_encoding:
            tgt = self.pos_encoder(tgt)
        output = self.transformer_decoder(tgt, memory)

        output = self.norm(output.mean(dim=1))
        output = self.dropout(output)
        return self.decoder(output)


class ProbSparseAttention(nn.Module):
    def __init__(self, d_model, nhead, factor):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, nhead, dropout=Config.dropout, batch_first=True)
        self.factor = factor

    def forward(self, q, k, v):
        attn_output, _ = self.attention(q, k, v)
        return attn_output


class InformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dropout, factor=5, use_positional_encoding=True):
        super().__init__()
        self.encoder = nn.Linear(input_size, d_model)
        self.use_positional_encoding = use_positional_encoding
        if use_positional_encoding:
            self.pos_encoder = PositionalEncoding(d_model)
        self.attn_layers = nn.ModuleList([
            ProbSparseAttention(d_model, nhead, factor) for _ in range(num_layers)
        ])

        decoder_layer = nn.TransformerDecoderLayer(
            d_model, nhead, dim_feedforward=d_model * 4, dropout=dropout, batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.decoder = nn.Linear(d_model, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.encoder(x)
        if self.use_positional_encoding:
            x = self.pos_encoder(x)
        for attn in self.attn_layers:
            x = attn(x, x, x)
        memory = x

        tgt = x
        if self.use_positional_encoding:
            tgt = self.pos_encoder(tgt)
        output = self.transformer_decoder(tgt, memory)

        output = self.norm(output.mean(dim=1))
        output = self.dropout(output)
        return self.decoder(output)


class Trainer:
    def __init__(self, model, model_name):
        self.model = model.to(Config.device)
        self.model_name = model_name
        self.criterion = nn.HuberLoss(delta=1.0)
        self.optimizer = optim.AdamW(model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
        self.warmup_scheduler = optim.lr_scheduler.LinearLR(self.optimizer, start_factor=0.1, total_iters=10)
        self.cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=Config.epochs - 10)
        self.best_loss = float('inf')
        self.early_stop_counter = 0

    def train_epoch(self, train_loader, epoch):
        self.model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(Config.device), y.to(Config.device)
            self.optimizer.zero_grad()
            outputs = self.model(X)
            loss = self.criterion(outputs.squeeze(), y)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            total_loss += loss.item()
        if epoch < 10:
            self.warmup_scheduler.step()
        else:
            self.cosine_scheduler.step()
        return total_loss / len(train_loader)

    def evaluate(self, test_loader):
        self.model.eval()
        total_loss = 0
        preds, truths = [], []
        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(Config.device), y.to(Config.device)
                outputs = self.model(X)
                loss = self.criterion(outputs.squeeze(), y)
                total_loss += loss.item()
                preds.extend(outputs.cpu().numpy().flatten())
                truths.extend(y.cpu().numpy().flatten())
        return total_loss / len(test_loader), np.array(preds), np.array(truths)

    def train(self, train_loader, test_loader):
        for epoch in range(Config.epochs):
            train_loss = self.train_epoch(train_loader, epoch)
            test_loss, preds, truths = self.evaluate(test_loader)
            if test_loss < self.best_loss:
                self.best_loss = test_loss
                torch.save(self.model.state_dict(), f'best_{self.model_name}.pth')
                self.early_stop_counter = 0
            else:
                self.early_stop_counter += 1
                if self.early_stop_counter >= 15:
                    print(f"Early stopping at epoch {epoch}")
                    break
            if epoch % 10 == 0:
                print(f"Epoch {epoch + 1}/{Config.epochs} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")
        self.model.load_state_dict(torch.load(f'best_{self.model_name}.pth'))
        return preds, truths


class Evaluator:
    @staticmethod
    def calculate_metrics(preds, truths):
        pred_dir = (preds > 0).astype(int)
        true_dir = (truths > 0).astype(int)
        fpr, tpr, thresholds = roc_curve(true_dir, preds)
        roc_auc = auc(fpr, tpr)

        metrics = {
            'RMSE': np.sqrt(mean_squared_error(truths, preds)),
            'MAE': mean_absolute_error(truths, preds),
            'Accuracy': accuracy_score(true_dir, pred_dir),
            'Recall': recall_score(true_dir, pred_dir),
            'Sharpe': np.mean(preds) / np.std(preds) * np.sqrt(252) if np.std(preds) != 0 else 0,
            'Correlation': np.corrcoef(truths, preds)[0, 1],
            'F1': f1_score(true_dir, pred_dir),
            'ConfusionMatrix': confusion_matrix(true_dir, pred_dir),
            'Precision': precision_score(true_dir, pred_dir),
            'AUC': roc_auc
        }
        return metrics


def main():
    processor = DataProcessor()
    results = {}
    for symbol in Config.symbols:
        print(f"\n=== Processing {symbol} ===")
        X, y = processor.process_data(symbol)
        if X is None or y is None:
            continue
        train_size = int(len(X) * Config.train_ratio)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
        test_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

        train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=Config.batch_size)

        model_results = {}
        input_size = X_train.shape[2]

        # 原始Transformer模型
        transformer = TransformerModel(input_size, **Config.model_config['Transformer'], dropout=Config.dropout, use_positional_encoding=True)
        trans_trainer = Trainer(transformer, 'Transformer')
        trans_preds, trans_truths = trans_trainer.train(train_loader, test_loader)
        model_results['Transformer'] = Evaluator.calculate_metrics(trans_preds, trans_truths)

        # 移除位置编码的Transformer模型
        transformer_no_pe = TransformerModel(input_size, **Config.model_config['Transformer'], dropout=Config.dropout, use_positional_encoding=False)
        trans_no_pe_trainer = Trainer(transformer_no_pe, 'Transformer_no_pe')
        trans_no_pe_preds, trans_no_pe_truths = trans_no_pe_trainer.train(train_loader, test_loader)
        model_results['Transformer_no_pe'] = Evaluator.calculate_metrics(trans_no_pe_preds, trans_no_pe_truths)

        # 原始Informer模型
        informer = InformerModel(input_size, **Config.model_config['Informer'], dropout=Config.dropout, use_positional_encoding=True)
        informer_trainer = Trainer(informer, 'Informer')
        informer_preds, informer_truths = informer_trainer.train(train_loader, test_loader)
        model_results['Informer'] = Evaluator.calculate_metrics(informer_preds, informer_truths)

        # 移除位置编码的Informer模型
        informer_no_pe = InformerModel(input_size, **Config.model_config['Informer'], dropout=Config.dropout, use_positional_encoding=False)
        informer_no_pe_trainer = Trainer(informer_no_pe, 'Informer_no_pe')
        informer_no_pe_preds, informer_no_pe_truths = informer_no_pe_trainer.train(train_loader, test_loader)
        model_results['Informer_no_pe'] = Evaluator.calculate_metrics(informer_no_pe_preds, informer_no_pe_truths)

        results[symbol] = model_results

        for model_name, preds in [('Transformer', trans_preds), ('Transformer_no_pe', trans_no_pe_preds), ('Informer', informer_preds), ('Informer_no_pe', informer_no_pe_preds)]:
            # 绘制预测结果
            plt.figure(figsize=(12, 6))
            plt.plot(trans_truths, label='True Returns', alpha=0.7)
            plt.plot(preds, label=f'{model_name} Predicted Returns', linestyle='--')
            plt.title(f'{symbol} Return Prediction - {model_name}')
            plt.legend()
            plt.savefig(f'{symbol}_prediction_ending5_{model_name}.png')
            plt.close()

            # 绘制ROC曲线
            pred_dir = (preds > 0).astype(int)
            true_dir = (trans_truths > 0).astype(int)
            fpr, tpr, thresholds = roc_curve(true_dir, preds)
            roc_auc = auc(fpr, tpr)
            plt.figure(figsize=(8, 8))
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'{symbol} {model_name} ROC Curve')
            plt.legend(loc="lower right")
            plt.savefig(f'{symbol}_roc_curve_{model_name}.png')
            plt.close()

    print("\n=== Final Results ===")
    for symbol in results:
        print(f"\n{symbol} Performance:")
        for model in results[symbol]:
            metrics = results[symbol][model]
            print(f"{model}:")
            print(f"  RMSE: {metrics['RMSE']:.4f} | MAE: {metrics['MAE']:.4f}")
            print(f"  Accuracy: {metrics['Accuracy']:.2%} | Recall: {metrics['Recall']:.2%}")
            print(f"  Sharpe: {metrics['Sharpe']:.2f} | Corr: {metrics['Correlation']:.2f}")
            print(f"  Precision: {metrics['Precision']:.2f}")
            print(f"  F1: {metrics['F1']:.2f}")
            print(f"  AUC: {metrics['AUC']:.2f}")
            print(f"  Confusion Matrix:\n{metrics['ConfusionMatrix']}")


if __name__ == '__main__':
    main()
    

[*********************100%***********************]  1 of 1 completed


=== Processing AAPL ===


Epoch 1/200 | Train Loss: 0.4284 | Test Loss: 0.5611
Epoch 11/200 | Train Loss: 0.0756 | Test Loss: 0.3253
Epoch 21/200 | Train Loss: 0.0440 | Test Loss: 0.1852
Epoch 31/200 | Train Loss: 0.0298 | Test Loss: 0.3306
Early stopping at epoch 35
Epoch 1/200 | Train Loss: 0.4478 | Test Loss: 0.8256
Epoch 11/200 | Train Loss: 0.2554 | Test Loss: 0.5544
Epoch 21/200 | Train Loss: 0.1204 | Test Loss: 0.5925
Early stopping at epoch 30
Epoch 1/200 | Train Loss: 0.4794 | Test Loss: 0.5608
Epoch 11/200 | Train Loss: 0.0711 | Test Loss: 0.1624
Epoch 21/200 | Train Loss: 0.0461 | Test Loss: 0.1612
Epoch 31/200 | Train Loss: 0.0425 | Test Loss: 0.1822
Epoch 41/200 | Train Loss: 0.0390 | Test Loss: 0.1820
Early stopping at epoch 47
Epoch 1/200 | Train Loss: 0.4137 | Test Loss: 0.5616
Epoch 11/200 | Train Loss: 0.2998 | Test Loss: 0.8074
Early stopping at epoch 15


[*********************100%***********************]  1 of 1 completed


=== Processing MSFT ===


...

Epoch 1/200 | Train Loss: 0.3763 | Test Loss: 0.6252
Epoch 11/200 | Train Loss: 0.0776 | Test Loss: 0.2042
Epoch 21/200 | Train Loss: 0.0449 | Test Loss: 0.1427
Epoch 31/200 | Train Loss: 0.0329 | Test Loss: 0.2309
Epoch 41/200 | Train Loss: 0.0235 | Test Loss: 0.2045
Early stopping at epoch 47
Epoch 1/200 | Train Loss: 0.3997 | Test Loss: 0.5561
Epoch 11/200 | Train Loss: 0.2660 | Test Loss: 0.5975
Epoch 21/200 | Train Loss: 0.1420 | Test Loss: 0.7466
Early stopping at epoch 22
Epoch 1/200 | Train Loss: 0.3786 | Test Loss: 0.6420
Epoch 11/200 | Train Loss: 0.0715 | Test Loss: 0.1613
Epoch 21/200 | Train Loss: 0.0467 | Test Loss: 0.1307
Early stopping at epoch 29
Epoch 1/200 | Train Loss: 0.3787 | Test Loss: 0.5407
Epoch 11/200 | Train Loss: 0.2889 | Test Loss: 0.4733
Epoch 21/200 | Train Loss: 0.2027 | Test Loss: 0.4299
Epoch 31/200 | Train Loss: 0.1677 | Test Loss: 0.4791
Early stopping at epoch 35

=== Processing NVDA ===


[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.3922 | Test Loss: 0.5912
Epoch 11/200 | Train Loss: 0.0756 | Test Loss: 0.2324
Epoch 21/200 | Train Loss: 0.0417 | Test Loss: 0.2256
Epoch 31/200 | Train Loss: 0.0265 | Test Loss: 0.1463
Epoch 41/200 | Train Loss: 0.0224 | Test Loss: 0.1490
Early stopping at epoch 47
Epoch 1/200 | Train Loss: 0.3787 | Test Loss: 0.6022
Epoch 11/200 | Train Loss: 0.2444 | Test Loss: 0.6612
Epoch 21/200 | Train Loss: 0.1038 | Test Loss: 0.5563
Epoch 31/200 | Train Loss: 0.0762 | Test Loss: 0.6300
Early stopping at epoch 33
Epoch 1/200 | Train Loss: 0.3907 | Test Loss: 0.5955
Epoch 11/200 | Train Loss: 0.0608 | Test Loss: 0.1686
Epoch 21/200 | Train Loss: 0.0459 | Test Loss: 0.1909
Epoch 31/200 | Train Loss: 0.0383 | Test Loss: 0.1967
Early stopping at epoch 31
Epoch 1/200 | Train Loss: 0.3690 | Test Loss: 0.5413
Epoch 11/200 | Train Loss: 0.2805 | Test Loss: 0.5603
Early stopping at epoch 15

=== Processing TSLA ===


[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.3839 | Test Loss: 0.4700
Epoch 11/200 | Train Loss: 0.0709 | Test Loss: 0.1893
Epoch 21/200 | Train Loss: 0.0413 | Test Loss: 0.1237
Epoch 31/200 | Train Loss: 0.0296 | Test Loss: 0.1225
Epoch 41/200 | Train Loss: 0.0237 | Test Loss: 0.1096
Epoch 51/200 | Train Loss: 0.0163 | Test Loss: 0.1355
Early stopping at epoch 54
Epoch 1/200 | Train Loss: 0.3818 | Test Loss: 0.4879
Epoch 11/200 | Train Loss: 0.2582 | Test Loss: 0.4908
Epoch 21/200 | Train Loss: 0.1290 | Test Loss: 0.5897
Epoch 31/200 | Train Loss: 0.0742 | Test Loss: 0.5611
Early stopping at epoch 38
Epoch 1/200 | Train Loss: 0.3966 | Test Loss: 0.5097
Epoch 11/200 | Train Loss: 0.0696 | Test Loss: 0.1387
Epoch 21/200 | Train Loss: 0.0434 | Test Loss: 0.1239
Epoch 31/200 | Train Loss: 0.0382 | Test Loss: 0.1106
Early stopping at epoch 36
Epoch 1/200 | Train Loss: 0.3881 | Test Loss: 0.4842
Epoch 11/200 | Train Loss: 0.2739 | Test Loss: 0.6647
Early stopping at epoch 16

=== Final Results ===

AAPL Per